# Cirq Conversion

In [ ]:
import cirq
from cirq.contrib.svg import SVGCircuit

from cirq_qubitization.quantum_graph.composite_bloq import CompositeBloq
from cirq_qubitization.jupyter_tools import show_bloq

## Using Cirq gates from Bloqs

 - `CirqGateAsBloq(gate)` lets you use any Cirq gate in your composite bloqs.
 - `CompositeBloq.from_cirq_circuit(circuit)` lets you turn any Cirq circuit into an equivalent `CompositeBloq`. Each subbloq will be a `CirqGateAsBloq` wrapper.

### `CirqGateAsBloq`

This wrapper uses the Cirq "calling convention" of one thru-register composted of a 1d array of qubits.

In [ ]:
from cirq_qubitization.quantum_graph.cirq_conversion import CirqGateAsBloq

cgab = CirqGateAsBloq(cirq.CNOT)
print(cgab)
for reg in cgab.registers:
    print(' ', reg)

In [ ]:
show_bloq(cgab)

### `CompositeBloq.from_cirq_circuit`

A Cirq circuit can be converted to a composite bloq by wrapping each operation with the `CirqGateAsBloq` wrapper. 

In [ ]:
# Make a random cirq circuit
qubits = cirq.LineQubit.range(4)
circuit = cirq.testing.random_circuit(qubits, n_moments=5, op_density=1.0, random_state=52)

# Convert to CompositeBloq
cbloq = CompositeBloq.from_cirq_circuit(circuit)
display(SVGCircuit(circuit))
display(show_bloq(cbloq))

### Unitaries

Both containers support numerical contraction to a dense unitary matrix. Cirq contracts operations into a large identity matrix "in order". Bloqs use `quimb` to find a good contraction ordering and perform the contraction. 

In [ ]:
import numpy as np

bloq_unitary = cbloq.tensor_contract()
cirq_unitary = circuit.unitary(qubits)
np.testing.assert_allclose(cirq_unitary, bloq_unitary, atol=1e-8)

### Back to `cirq.Circuit`

In [ ]:
# Note: a 1d `wireshape` bloq register is actually two-dimensional in cirq-world
# because of the implicit `bitsize` dimension (which must be explicit in cirq-world).
# CirqGateAsBloq has registers of bitsize=1 and wireshape=(n,); hence the list transpose below.
circuit2 = cbloq.to_cirq_circuit({'qubits': [[q] for q in qubits]})
SVGCircuit(circuit2)

In [ ]:
# We lose the moment structure during the roundtrip.
circuit == circuit2

In [ ]:
# But the left-aligned `circuit` is recovered.
cirq.Circuit(circuit.all_operations()) == circuit2

## Converting Bloqs to Cirq objects

 - `Bloq.as_cirq_op` is an overridable method to declare what cirq operation corresponds to a bloq.
 - `CompositeBloq.to_cirq_circuit` will export a CompositeBloq to a cirq circuit.
 - `BloqAsCirqGate` provides a shim for using bloqs in cirq circuits automatically.

### `as_cirq_op` 

Bloqs can override `as_cirq_op` to optionally declare their corresponding Cirq operation.

In [ ]:
# TODO: show a bloq definition

### `CompositeBloq.to_cirq_circuit`

A composite bloq can be turned into a circuit composed of the result of `as_cirq_op` for each of the subbloqs.

In [ ]:
# TODO: show a composite bloq

### `BloqAsCirqGate`

The default behavior of `as_cirq_op` will shim the bloq into this object which lets you use a bloq in cirq circuits.

In [ ]:
# TODO: show example

### Allocation and de-allocation

TODO: describe

In [ ]:
from cirq_qubitization.bloq_algos.and_bloq import MultiAnd

multi_and = MultiAnd(cvs=(1, 1, 1, 1))

In [ ]:
cirq_quregs = multi_and.registers.get_cirq_quregs()
cirq_quregs

In [ ]:
multi_and_circuit = multi_and.decompose_bloq().to_cirq_circuit(cirq_quregs)
SVGCircuit(multi_and_circuit)

In [ ]:
# Note: our dictionary is mutated. We can figure out the
# identity of the allocated qubits.
cirq_quregs